In [1]:
from pathlib import Path
from models.transfersdataset import TransfersDataset
from models.lightgbm import LightGBM as Model

In [2]:
import pandas as pd
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
td.encode_last_positions()
td.encode()
td.drop()

In [5]:
mid_features = ['succssdefact90','defduels90','defduelswpct','aerduels90','aerduelspct','sltackles90','padjsltackles','shotblocks90',
                'intrcpt90','padjintrcpt','fouls90','goals90','xg90','assists90','crosses90','acccrossespct','crossesleft90',
                'acccrossesleftpct','crossesright90','acccrossesrightpct','crosses6yard90','dribbles90','succssdribblespct','offduels90',
                'offduelspct','touchesbox90','prgruns90','acclr90','rcvdpasses90','rcvdlongpasses90','foulsa90','passes90',
                'accpassespct','fwdpasses90','accfwdpassespct','bpasses90','accbpassespct','latpasses90','acclatpassespct','shortmedpasses90',
                'accshortmedpassespct','longpasses90','acclongpassesct','avgpasslen','avglongpasslen','xa90','shotassist90','secassist90',
                'thirdassist90','smartpasses90','accsmartpassespct','kp90','final3rdpasses90','accfinal3rdpassespct','penareapasses90',
                'accpenareapassespct','thrpasses90','accthrpassespct','deepcomp90','deepcompcrosses90','prpasses90','accprpassespct',
                'directfk90','directfkontrgtpct','corners90','pens','pencnvrspct']
mid_features2 = ['assists90', 'succssdribblespct', 'rcards90', 'hdrgoals90', 'pens', 'accthrpassespct', 'defduelswpct', 'directfkontrgtpct', 'goalcnvrsnpct', 'fwdpasses90', 'final3rdpasses90', 'accprpassespct', 
                 'accsmartpassespct', 'sattact90', 'acclr90', 'penareapasses90']
mid_features3 = ['goals90', 'xg90','assists90', 'xa90', 'kp90', 'thrpasses90', 'final3rdpasses90', 'penareapasses90']

In [6]:
scores = pd.read_csv('lightgbm_mid.csv').iloc[:,1:].sort_values('MSE')
scores.Features = scores.Features.str.split(', ')
feature_sets = scores.Features.head(3).to_dict()

In [7]:
model_scores = pd.DataFrame()
def pipeline(pos, features=None, full_feature_set=False):
    td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
    td.filter_postion(pos)
    td.encode()
    td.drop()
    m = Model(data = td.data, features = features, full_feature_set=full_feature_set)
    display(m.data[m.features], features)
    m.train_test_split()
    m.scale()
    m.tune_hp()
    m.train()
    m.inverse_scale()
    global model_scores
    model_scores = pd.concat([model_scores, m.score(thresh=1.5)])
    # display(m.feature_importance())
    m.plot_predictions()
    display(m.top_n_predictions(20))
    display(m.top_n_predictions(20, worst=True))
    m.calculate_effectiveness()

In [8]:
pipeline('MID')

,age,season,window,fee,club_from_elo,club_to_elo,league_from_elo,league_to_elo,marketval,matchesplayed,...,deepcomp90,deepcompcrosses90,prpasses90,accprpassespct,fk90,directfk90,directfkontrgtpct,corners90,pens,pencnvrspct
2,26,0.0,1,87.0,1743,1842,1696.800000,1696.800000,50.0,31,...,0.49,0.00,9.71,82.33,0.00,0.0,0.00,0.00,0,0.0
3,22,0.0,1,86.0,1816,2000,1429.238095,1671.233333,85.0,31,...,2.01,0.27,13.61,88.97,0.03,0.0,0.00,0.10,0,0.0
5,23,0.0,1,80.0,1877,1947,1671.233333,1605.100000,70.0,14,...,0.15,0.46,6.97,79.12,0.00,0.0,0.00,0.00,0,0.0
6,23,0.0,1,70.0,1877,2041,1671.233333,1696.800000,80.0,34,...,0.35,0.00,7.74,82.93,0.03,0.0,0.00,0.00,0,0.0
9,22,0.0,1,62.0,1731,1895,1544.166667,1696.800000,65.0,34,...,1.23,0.13,10.44,90.36,0.03,0.0,0.00,0.00,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2216,24,4.0,0,8.5,1608,1759,1630.900000,1630.900000,6.0,33,...,0.09,0.03,6.02,62.63,0.03,0.0,0.00,0.00,0,0.0
2217,25,4.0,0,2.7,1608,1621,1630.900000,1653.333333,3.0,9,...,0.16,0.79,6.81,81.40,0.00,0.0,0.00,0.00,0,0.0
2218,25,4.0,0,2.7,1608,1621,1630.900000,1653.333333,3.0,16,...,0.19,0.06,5.95,69.15,0.00,0.0,0.00,0.00,0,0.0
2220,22,4.0,0,2.5,1524,1569,1474.736842,1552.633333,1.8,29,...,2.20,0.29,3.15,86.05,0.00,0.0,0.00,0.00,0,0.0


None

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'num_leaves': 2, 'learning_rate': 0.06761588387128255, 'subsample': 0.615284416202518, 'colsample_bytree': 0.7506464183774275, 'min_child_samples': 9, 'reg_alpha': 2.002229058209321, 'reg_lambda': 6.513304815119558}
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009353 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14644
[LightGBM] [Info] Number of data points in the train set: 941, number of used features: 91
thresh = 1.5
mae: 3.6376796962112055
mape: 0.8577116091023879
mae_below_thresh: 2.4472962504413185
mape_above_thresh: 0.45163803653240103


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
1738,Andreas Schjelderup,18,22/23,Denmark,DK1,FC Nordsjaelland,Portugal,PO1,SL Benfica,0,4.0,9.0,9.00,9.0,8.987277,0.012723
1433,Cyle Larin,27,22/23,Türkiye,TR1,Besiktas JK,Belgium,BE1,Club Brugge KV,1,10.0,8.0,0.00,8.0,8.020399,0.020399
1086,Mame Thiam,28,21/22,Türkiye,TR1,Fenerbahce,Türkiye,TR1,Kayserispor,1,2.5,3.0,1.50,3.0,2.935898,0.064102
1627,Brian Oliván,28,22/23,Spain,ES1,RCD Mallorca,Spain,ES1,RCD Espanyol Barcelona,1,2.5,6.0,0.00,6.0,5.932420,0.067580
1070,Rubén Rochina,30,21/22,Spain,ES1,Levante UD,Spain,ES1,Granada CF,1,2.8,2.5,0.00,2.5,2.639601,0.139601
633,Maxime Gonalons,31,20/21,Italy,IT1,AS Roma,Spain,ES1,Granada CF,1,2.8,2.5,4.00,2.5,2.639601,0.139601
1934,Jonas Hofmann,30,23/24,Germany,L1,Borussia Mönchengladbach,Germany,L1,Bayer 04 Leverkusen,1,13.0,13.0,10.00,13.0,13.154555,0.154555
1486,Marc Bartra,31,22/23,Spain,ES1,Real Betis Balompié,Türkiye,TR1,Trabzonspor,1,6.0,3.7,1.25,3.7,3.880660,0.180660
106,Eljif Elmas,19,19/20,Türkiye,TR1,Fenerbahce,Italy,IT1,SSC Napoli,1,10.0,15.5,17.70,15.5,15.801583,0.301583
1511,Lincoln,23,22/23,Portugal,PO1,CD Santa Clara,Türkiye,TR1,Fenerbahce,1,5.0,5.5,3.50,5.5,5.823589,0.323589


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
121,Diego Carlos,26,19/20,France,FR1,FC Nantes,Spain,ES1,Sevilla FC,1,13.0,50.0,15.0,50.0,14.425761,35.574239
122,Lucas Ocampos,24,19/20,France,FR1,Olympique Marseille,Spain,ES1,Sevilla FC,1,15.0,50.0,15.0,50.0,15.028542,34.971458
854,David Alaba,29,21/22,Germany,L1,Bayern Munich,Spain,ES1,Real Madrid,1,55.0,55.0,0.0,55.0,32.681647,22.318353
496,Pierre-Emile Højbjerg,25,20/21,England,GB1,Southampton FC,England,GB1,Tottenham Hotspur,1,20.0,40.0,16.6,40.0,18.974456,21.025544
452,Arthur Melo,24,20/21,Spain,ES1,FC Barcelona,Italy,IT1,Juventus FC,1,56.0,40.0,80.6,40.0,59.408682,19.408682
1803,Moisés Caicedo,21,23/24,England,GB1,Brighton & Hove Albion,England,GB1,Chelsea FC,1,75.0,90.0,116.0,90.0,70.691419,19.308581
96,Christopher Nkunku,21,19/20,France,FR1,Paris Saint-Germain,Germany,L1,RB Leipzig,1,12.0,33.0,19.5,33.0,15.317688,17.682312
2048,Yangel Herrera,25,23/24,England,GB1,Manchester City,Spain,ES1,Girona FC,1,5.0,20.0,5.0,20.0,5.061836,14.938164
1307,Yves Bissouma,25,22/23,England,GB1,Brighton & Hove Albion,England,GB1,Tottenham Hotspur,1,35.0,18.0,29.2,18.0,32.597787,14.597787
113,Douglas Luiz,21,19/20,England,GB1,Manchester City,England,GB1,Aston Villa,1,2.5,22.0,16.8,22.0,8.273970,13.726030


In [9]:
pipeline('MID', mid_features)

,age,season,window,fee,club_from_elo,club_to_elo,league_from_elo,league_to_elo,marketval,matchesplayed,...,accthrpassespct,deepcomp90,deepcompcrosses90,prpasses90,accprpassespct,directfk90,directfkontrgtpct,corners90,pens,pencnvrspct
2,26,0.0,1,87.0,1743,1842,1696.800000,1696.800000,50.0,31,...,27.27,0.49,0.00,9.71,82.33,0.0,0.00,0.00,0,0.0
3,22,0.0,1,86.0,1816,2000,1429.238095,1671.233333,85.0,31,...,44.12,2.01,0.27,13.61,88.97,0.0,0.00,0.10,0,0.0
5,23,0.0,1,80.0,1877,1947,1671.233333,1605.100000,70.0,14,...,0.00,0.15,0.46,6.97,79.12,0.0,0.00,0.00,0,0.0
6,23,0.0,1,70.0,1877,2041,1671.233333,1696.800000,80.0,34,...,37.50,0.35,0.00,7.74,82.93,0.0,0.00,0.00,0,0.0
9,22,0.0,1,62.0,1731,1895,1544.166667,1696.800000,65.0,34,...,34.78,1.23,0.13,10.44,90.36,0.0,0.00,0.00,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2216,24,4.0,0,8.5,1608,1759,1630.900000,1630.900000,6.0,33,...,28.57,0.09,0.03,6.02,62.63,0.0,0.00,0.00,0,0.0
2217,25,4.0,0,2.7,1608,1621,1630.900000,1653.333333,3.0,9,...,0.00,0.16,0.79,6.81,81.40,0.0,0.00,0.00,0,0.0
2218,25,4.0,0,2.7,1608,1621,1630.900000,1653.333333,3.0,16,...,26.67,0.19,0.06,5.95,69.15,0.0,0.00,0.00,0,0.0
2220,22,4.0,0,2.5,1524,1569,1474.736842,1552.633333,1.8,29,...,36.84,2.20,0.29,3.15,86.05,0.0,0.00,0.00,0,0.0


['succssdefact90',
 'defduels90',
 'defduelswpct',
 'aerduels90',
 'aerduelspct',
 'sltackles90',
 'padjsltackles',
 'shotblocks90',
 'intrcpt90',
 'padjintrcpt',
 'fouls90',
 'goals90',
 'xg90',
 'assists90',
 'crosses90',
 'acccrossespct',
 'crossesleft90',
 'acccrossesleftpct',
 'crossesright90',
 'acccrossesrightpct',
 'crosses6yard90',
 'dribbles90',
 'succssdribblespct',
 'offduels90',
 'offduelspct',
 'touchesbox90',
 'prgruns90',
 'acclr90',
 'rcvdpasses90',
 'rcvdlongpasses90',
 'foulsa90',
 'passes90',
 'accpassespct',
 'fwdpasses90',
 'accfwdpassespct',
 'bpasses90',
 'accbpassespct',
 'latpasses90',
 'acclatpassespct',
 'shortmedpasses90',
 'accshortmedpassespct',
 'longpasses90',
 'acclongpassesct',
 'avgpasslen',
 'avglongpasslen',
 'xa90',
 'shotassist90',
 'secassist90',
 'thirdassist90',
 'smartpasses90',
 'accsmartpassespct',
 'kp90',
 'final3rdpasses90',
 'accfinal3rdpassespct',
 'penareapasses90',
 'accpenareapassespct',
 'thrpasses90',
 'accthrpassespct',
 'deepcom

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'num_leaves': 38, 'learning_rate': 0.051853151979768176, 'subsample': 0.8441658139944456, 'colsample_bytree': 0.7063229483275327, 'min_child_samples': 6, 'reg_alpha': 1.0955918210125994, 'reg_lambda': 2.228647736397141}
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003100 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13156
[LightGBM] [Info] Number of data points in the train set: 941, number of used features: 80
thresh = 1.5
mae: 3.3413161047414848
mape: 0.5666123718961715
mae_below_thresh: 1.1765565716973998
mape_above_thresh: 0.41124447280837567


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
419,Sung-yueng Ki,31,19/20,England,GB1,Newcastle United,Spain,ES1,RCD Mallorca,0,6.0,2.4,0.0,2.4,2.429307,0.029307
1356,Moussa Niakhaté,26,22/23,Germany,L1,1.FSV Mainz 05,England,GB1,Nottingham Forest,1,18.0,18.0,10.0,18.0,18.058740,0.058740
520,Ibrahima Diallo,21,20/21,France,FR1,Stade Brestois 29,England,GB1,Southampton FC,1,9.0,15.0,12.0,15.0,14.926884,0.073116
1367,Marcos Senesi,25,22/23,Netherlands,NL1,Feyenoord Rotterdam,England,GB1,AFC Bournemouth,1,17.0,22.0,15.0,22.0,21.916494,0.083506
1633,Jure Balkovec,27,22/23,Türkiye,TR1,Fatih Karagümrük,Türkiye,TR1,Alanyaspor,1,2.3,2.2,0.0,2.2,2.109174,0.090826
1920,Callum Hudson-Odoi,22,23/24,England,GB1,Chelsea FC,England,GB1,Nottingham Forest,1,15.0,12.0,3.5,12.0,12.101006,0.101006
2132,Kristoffer Olsson,28,23/24,Belgium,BE1,RSC Anderlecht,Denmark,DK1,FC Midtjylland,1,2.5,2.5,2.8,2.5,2.613286,0.113286
2175,Christophe Hérelle,31,23/24,France,FR1,Stade Brestois 29,France,FR1,FC Metz,1,2.0,1.8,0.0,1.8,1.677746,0.122254
1388,Orel Mangala,24,22/23,Germany,L1,VfB Stuttgart,England,GB1,Nottingham Forest,1,15.0,16.0,13.0,16.0,16.135626,0.135626
1064,Valère Germain,31,21/22,France,FR1,Olympique Marseille,France,FR1,Montpellier HSC,1,3.0,2.5,0.0,2.5,2.667600,0.167600


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
121,Diego Carlos,26,19/20,France,FR1,FC Nantes,Spain,ES1,Sevilla FC,1,13.0,50.0,15.0,50.0,15.250182,34.749818
122,Lucas Ocampos,24,19/20,France,FR1,Olympique Marseille,Spain,ES1,Sevilla FC,1,15.0,50.0,15.0,50.0,17.493553,32.506447
854,David Alaba,29,21/22,Germany,L1,Bayern Munich,Spain,ES1,Real Madrid,1,55.0,55.0,0.0,55.0,27.492674,27.507326
1812,Sandro Tonali,23,23/24,Italy,IT1,AC Milan,England,GB1,Newcastle United,1,50.0,40.0,64.0,40.0,60.235394,20.235394
496,Pierre-Emile Højbjerg,25,20/21,England,GB1,Southampton FC,England,GB1,Tottenham Hotspur,1,20.0,40.0,16.6,40.0,23.015436,16.984564
452,Arthur Melo,24,20/21,Spain,ES1,FC Barcelona,Italy,IT1,Juventus FC,1,56.0,40.0,80.6,40.0,56.646033,16.646033
1307,Yves Bissouma,25,22/23,England,GB1,Brighton & Hove Albion,England,GB1,Tottenham Hotspur,1,35.0,18.0,29.2,18.0,34.393830,16.393830
463,Donny van de Beek,23,20/21,Netherlands,NL1,Ajax Amsterdam,England,GB1,Manchester United,1,44.0,30.0,39.0,30.0,45.813786,15.813786
96,Christopher Nkunku,21,19/20,France,FR1,Paris Saint-Germain,Germany,L1,RB Leipzig,1,12.0,33.0,19.5,33.0,17.740786,15.259214
1811,Mason Mount,24,23/24,England,GB1,Chelsea FC,England,GB1,Manchester United,1,60.0,45.0,64.2,45.0,59.810043,14.810043


In [10]:
scores = pd.read_csv('lightgbm_mid.csv').iloc[:,1:].sort_values('MSE')
scores.Features = scores.Features.str.split(', ')
feature_sets = scores.Features.head(10).to_dict()

In [11]:
for _, features in feature_sets.items():
    pipeline(pos='MID', features=features, full_feature_set=True)

,age,season,window,fee,club_from_elo,club_to_elo,league_from_elo,league_to_elo,marketval,matchesplayed,...,accprpassespct,ontrgtpct,deepcompcrosses90,padjintrcpt,smartpasses90,accsmartpassespct,succssdefact90,goalcnvrsnpct,penareapasses90,dribbles90
2,26,0.0,1,87.0,1743,1842,1696.800000,1696.800000,50.0,31,...,82.33,45.00,0.00,7.84,0.42,38.46,9.58,15.000,0.74,0.87
3,22,0.0,1,86.0,1816,2000,1429.238095,1671.233333,85.0,31,...,88.97,39.29,0.27,9.51,1.02,53.33,9.86,14.286,2.90,3.51
5,23,0.0,1,80.0,1877,1947,1671.233333,1605.100000,70.0,14,...,79.12,20.00,0.46,5.57,0.54,28.57,10.72,20.000,0.92,1.53
6,23,0.0,1,70.0,1877,2041,1671.233333,1696.800000,80.0,34,...,82.93,28.00,0.00,6.72,0.38,41.67,11.29,12.000,0.53,1.38
9,22,0.0,1,62.0,1731,1895,1544.166667,1696.800000,65.0,34,...,90.36,23.33,0.13,5.78,1.57,38.00,8.08,3.333,2.36,4.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2216,24,4.0,0,8.5,1608,1759,1630.900000,1630.900000,6.0,33,...,62.63,0.00,0.03,5.87,0.09,33.33,13.53,0.000,0.40,1.06
2217,25,4.0,0,2.7,1608,1621,1630.900000,1653.333333,3.0,9,...,81.40,40.00,0.79,4.67,0.16,0.00,7.92,0.000,1.90,2.38
2218,25,4.0,0,2.7,1608,1621,1630.900000,1653.333333,3.0,16,...,69.15,0.00,0.06,7.69,0.19,0.00,8.93,0.000,0.76,0.76
2220,22,4.0,0,2.5,1524,1569,1474.736842,1552.633333,1.8,29,...,86.05,41.67,0.29,3.90,1.47,35.00,6.38,12.500,2.64,8.00


['age',
 'season',
 'window',
 'fee',
 'club_from_elo',
 'club_to_elo',
 'league_from_elo',
 'league_to_elo',
 'marketval',
 'matchesplayed',
 'minsplayed',
 'foot',
 'height',
 'weight',
 'crossesleft90',
 'foulsa90',
 'aerduelspct',
 'npgoals90',
 'pens',
 'rcards90',
 'pencnvrspct',
 'thirdassist90',
 'hdrgoals90',
 'directfkontrgtpct',
 'directfk90',
 'fouls90',
 'assists90',
 'shotassist90',
 'xg90',
 'goals90',
 'accprpassespct',
 'ontrgtpct',
 'deepcompcrosses90',
 'padjintrcpt',
 'smartpasses90',
 'accsmartpassespct',
 'succssdefact90',
 'goalcnvrsnpct',
 'penareapasses90',
 'dribbles90']

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'num_leaves': 30, 'learning_rate': 0.033566130320162764, 'subsample': 0.6406349326724434, 'colsample_bytree': 0.5013869799056277, 'min_child_samples': 6, 'reg_alpha': 0.06661662434026883, 'reg_lambda': 0.09436864355144078}
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004717 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4294
[LightGBM] [Info] Number of data points in the train set: 941, number of used features: 39
thresh = 1.5
mae: 3.468805999290132
mape: 0.6610446208195502
mae_below_thresh: 1.492295067331161
mape_above_thresh: 0.4484542665382176


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
1615,Marcus Ingvartsen,26,22/23,Germany,L1,1.FC Union Berlin,Germany,L1,1.FSV Mainz 05,1,2.50,5.5,2.30,5.5,5.500699,0.000699
1711,Levent Mercan,21,22/23,Germany,L1,FC Schalke 04,Türkiye,TR1,Fatih Karagümrük,1,1.20,2.5,0.00,2.5,2.521880,0.021880
316,Ryota Morioka,28,19/20,Belgium,BE1,RSC Anderlecht,Belgium,BE1,R Charleroi SC,1,2.20,2.4,1.50,2.4,2.428965,0.028965
2137,Emre Kılınç,28,23/24,Türkiye,TR1,Galatasaray,Türkiye,TR1,Samsunspor,1,2.80,2.3,0.00,2.3,2.249908,0.050092
938,João Mário,28,21/22,Italy,IT1,Inter Milan,Portugal,PO1,SL Benfica,1,12.00,10.0,0.00,10.0,10.085454,0.085454
1052,Deniz Türüç,28,21/22,Türkiye,TR1,Fenerbahce,Türkiye,TR1,Basaksehir FK,1,3.10,2.7,0.00,2.7,2.599050,0.100950
1511,Lincoln,23,22/23,Portugal,PO1,CD Santa Clara,Türkiye,TR1,Fenerbahce,1,5.00,5.5,3.50,5.5,5.618402,0.118402
2175,Christophe Hérelle,31,23/24,France,FR1,Stade Brestois 29,France,FR1,FC Metz,1,2.00,1.8,0.00,1.8,1.962417,0.162417
1633,Jure Balkovec,27,22/23,Türkiye,TR1,Fatih Karagümrük,Türkiye,TR1,Alanyaspor,1,2.30,2.2,0.00,2.2,2.376000,0.176000
1585,Matías Vecino,30,22/23,Italy,IT1,Inter Milan,Italy,IT1,SS Lazio,1,3.50,4.5,0.00,4.5,4.709400,0.209400


...

['age',
 'season',
 'window',
 'fee',
 'club_from_elo',
 'club_to_elo',
 'league_from_elo',
 'league_to_elo',
 'marketval',
 'matchesplayed',
 'minsplayed',
 'foot',
 'height',
 'weight',
 'crossesleft90',
 'foulsa90',
 'aerduelspct',
 'npgoals90',
 'pens',
 'rcards90',
 'pencnvrspct',
 'thirdassist90',
 'hdrgoals90']

  0%|          | 0/50 [00:00<?, ?it/s]

Best parameters: {'num_leaves': 22, 'learning_rate': 0.06483761061890242, 'subsample': 0.7928173571843319, 'colsample_bytree': 0.74673967564493, 'min_child_samples': 12, 'reg_alpha': 7.977614217779266, 'reg_lambda': 3.0553929401858753}
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003281 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1860
[LightGBM] [Info] Number of data points in the train set: 941, number of used features: 22
thresh = 1.5
mae: 3.493924638886689
mape: 0.6458312132868602
mae_below_thresh: 1.6183849411743185
mape_above_thresh: 0.4023844153100554


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
1511,Lincoln,23,22/23,Portugal,PO1,CD Santa Clara,Türkiye,TR1,Fenerbahce,1,5.0,5.5,3.50,5.5,5.495566,0.004434
1550,Michal Sadílek,23,22/23,Netherlands,NL1,PSV Eindhoven,Netherlands,NL1,Twente Enschede FC,1,4.0,4.0,1.50,4.0,4.011408,0.011408
2209,Ylber Ramadani,27,23/24,Scotland,SC1,Aberdeen FC,Italy,IT1,US Lecce,1,0.8,3.8,1.20,3.8,3.835743,0.035743
775,Castro,32,20/21,Türkiye,TR1,Göztepe,Portugal,PO1,SC Braga,1,1.0,1.5,0.00,1.5,1.552055,0.052055
1616,Rubén Alcaraz,31,22/23,Spain,ES1,Real Valladolid CF,Spain,ES1,Cádiz CF,1,2.5,2.0,1.00,2.0,2.061541,0.061541
419,Sung-yueng Ki,31,19/20,England,GB1,Newcastle United,Spain,ES1,RCD Mallorca,0,6.0,2.4,0.00,2.4,2.338394,0.061606
2157,Mads Bech Sörensen,24,23/24,England,GB1,Brentford FC,Denmark,DK1,FC Midtjylland,1,2.5,2.5,0.00,2.5,2.570979,0.070979
1356,Moussa Niakhaté,26,22/23,Germany,L1,1.FSV Mainz 05,England,GB1,Nottingham Forest,1,18.0,18.0,10.00,18.0,18.074889,0.074889
1633,Jure Balkovec,27,22/23,Türkiye,TR1,Fatih Karagümrük,Türkiye,TR1,Alanyaspor,1,2.3,2.2,0.00,2.2,2.076032,0.123968
1585,Matías Vecino,30,22/23,Italy,IT1,Inter Milan,Italy,IT1,SS Lazio,1,3.5,4.5,0.00,4.5,4.374695,0.125305


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error
121,Diego Carlos,26,19/20,France,FR1,FC Nantes,Spain,ES1,Sevilla FC,1,13.0,50.0,15.0,50.0,14.580385,35.419615
122,Lucas Ocampos,24,19/20,France,FR1,Olympique Marseille,Spain,ES1,Sevilla FC,1,15.0,50.0,15.0,50.0,15.435128,34.564872
854,David Alaba,29,21/22,Germany,L1,Bayern Munich,Spain,ES1,Real Madrid,1,55.0,55.0,0.0,55.0,28.856549,26.143451
1812,Sandro Tonali,23,23/24,Italy,IT1,AC Milan,England,GB1,Newcastle United,1,50.0,40.0,64.0,40.0,62.071407,22.071407
496,Pierre-Emile Højbjerg,25,20/21,England,GB1,Southampton FC,England,GB1,Tottenham Hotspur,1,20.0,40.0,16.6,40.0,20.979137,19.020863
1307,Yves Bissouma,25,22/23,England,GB1,Brighton & Hove Albion,England,GB1,Tottenham Hotspur,1,35.0,18.0,29.2,18.0,36.502632,18.502632
1803,Moisés Caicedo,21,23/24,England,GB1,Brighton & Hove Albion,England,GB1,Chelsea FC,1,75.0,90.0,116.0,90.0,71.575012,18.424988
463,Donny van de Beek,23,20/21,Netherlands,NL1,Ajax Amsterdam,England,GB1,Manchester United,1,44.0,30.0,39.0,30.0,48.224745,18.224745
1811,Mason Mount,24,23/24,England,GB1,Chelsea FC,England,GB1,Manchester United,1,60.0,45.0,64.2,45.0,62.269122,17.269122
96,Christopher Nkunku,21,19/20,France,FR1,Paris Saint-Germain,Germany,L1,RB Leipzig,1,12.0,33.0,19.5,33.0,16.002615,16.997385


In [12]:
with pd.option_context('display.max_colwidth', None):
    display(model_scores.sort_values('mape_above_thresh'))

,mae,mape,mae_below_thresh,mape_above_thresh,features
0,3.215333,0.536748,1.174966,0.374980,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, crossesleft90, foulsa90, aerduelspct, npgoals90, pens, rcards90, pencnvrspct]"
0,3.359855,0.556566,1.208970,0.379997,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, crossesleft90, foulsa90, aerduelspct, npgoals90, pens, rcards90, pencnvrspct, thirdassist90, hdrgoals90, directfkontrgtpct, directfk90, fouls90, assists90, shotassist90, xg90, goals90, accprpassespct, ontrgtpct, deepcompcrosses90, padjintrcpt, smartpasses90, accsmartpassespct, succssdefact90, goalcnvrsnpct]"
0,3.316533,0.545205,1.152432,0.390145,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, crossesleft90, foulsa90, aerduelspct, npgoals90, pens, rcards90, pencnvrspct, thirdassist90, hdrgoals90, directfkontrgtpct, directfk90, fouls90, assists90, shotassist90, xg90, goals90, accprpassespct, ontrgtpct, deepcompcrosses90, padjintrcpt, smartpasses90, accsmartpassespct, succssdefact90, goalcnvrsnpct, penareapasses90, dribbles90, rcvdlongpasses90]"
0,3.295891,0.597989,1.408866,0.393287,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, crossesleft90, foulsa90, aerduelspct, npgoals90, pens, rcards90]"
0,3.388635,0.640726,1.604845,0.396975,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, crossesleft90, foulsa90, aerduelspct, npgoals90, pens, rcards90, pencnvrspct, thirdassist90, hdrgoals90, directfkontrgtpct, directfk90, fouls90, assists90, shotassist90, xg90, goals90, accprpassespct, ontrgtpct, deepcompcrosses90, padjintrcpt, smartpasses90, accsmartpassespct]"
0,3.493925,0.645831,1.618385,0.402384,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, crossesleft90, foulsa90, aerduelspct, npgoals90, pens, rcards90, pencnvrspct, thirdassist90, hdrgoals90]"
0,3.315733,0.519373,0.932517,0.410964,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, crossesleft90, foulsa90, aerduelspct, npgoals90, pens, rcards90, pencnvrspct, thirdassist90, hdrgoals90, directfkontrgtpct, directfk90, fouls90, assists90, shotassist90, xg90, goals90, accprpassespct, ontrgtpct, deepcompcrosses90, padjintrcpt, smartpasses90, accsmartpassespct, succssdefact90]"
0,3.341316,0.566612,1.176557,0.411244,"[age, season, window, fee, club_from_elo, club_to_elo, league_from_elo, league_to_elo, marketval, matchesplayed, minsplayed, foot, height, weight, succssdefact90, defduels90, defduelswpct, aerduels90, aerduelspct, sltackles90, padjsltackles, shotblocks90, intrcpt90, padjintrcpt, fouls90, goals90, xg90, assists90, crosses90, acccrossespct, crossesleft90, acccrossesleftpct, crossesright90, acccrossesrightpct, crosses6yard90, dribbles90, succssdribblespct, offduels90, offduelspct, touchesbox90, prgruns90, acclr90, rcvdpasses90, rcvdlongpasses90, foulsa90, passes90, accpassespct, fwdpasses90, accfwdpassespct, bpasses90, accbpassespct, latpasses90, acclatpassespct, shortmedpasses90, accshortmedpassespct, longpasses90, acclongpassesct, avgpasslen, avglongpasslen, xa90, shotassist90, secassist90, thirdassist90, smartpasses90, accsmartpassespct, kp90, final3rdpasses90, accfinal3rdpassespct, penareapasses90, accpenareapassespct, thrpasses90, accthrpassespct, deepcomp90, deepcompcrosses90, prpasses90, accprpassespct, directfk90, directfkontrgtpct, corners90, pens, pencnvrspct]"
0,3.575974,0.754467,2.068584